<img src="images/ragna-logo.png" width=20% align="right"/>

# RAG and LLM Experiments

<hr>

## Explore the Web UI

Open a terminal window and run the following

1. Activate the conda environment
   
```bash
conda activate global-pycon-de
```

2. Start the UI

```bash
PYTHON_PATH="${HOME}:${PYTHON_PATH}" \
  dotenv --file ~/shared/analyst/.env run -- \
    python -m \
        ragna ui --config ragna.toml
```

3. Go to https://pycon-tutorial.quansight.dev/user/{USER}/proxy/31477/ (replace {USER} with your username)

### Aside: Local machine

On your personal computers, you can directly run: `ragna ui` to start the UI

## Advanced configuration

`rag.chat()` takes the following keyword arguments:

* `chunk_size`
* `chunk_overlap`
* `num_tokens`

## Compare LLMs

In [1]:
import asyncio
import itertools
from pathlib import Path
from pprint import pprint

from dotenv import load_dotenv
from local_llm import Mistral7BInstruct
from ragna import Rag
from ragna.assistants import Gpt4, Gpt35Turbo16k
from ragna.source_storages import Chroma

In [7]:
documents = [
    "files/psf-report-2022.pdf",
    "files/psf-report-2021.pdf",
]

source_storages = [Chroma]
assistants = [Gpt35Turbo16k, Gpt4, Mistral7BInstruct()]

prompt = "What was PSF's net income in 2021 and 2022?"

In [3]:
dotenv_path = Path.home() / Path("shared/analyst/api-keys.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [8]:
rag = Rag()


async def answer_prompt(source_storage, assistant):
    async with rag.chat(
        documents=documents,
        source_storage=source_storage,
        assistant=assistant,
    ) as chat:
        message = await chat.answer(prompt)
        return message.content

In [9]:
experiments = {
    (source_storage.display_name(), assistant.display_name()): answer_prompt(
        source_storage, assistant
    )
    for source_storage, assistant in itertools.product(source_storages, assistants)
}

pprint(experiments)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7c0885f26240>,
 ('Chroma', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7c0885fdcf40>,
 ('Chroma', 'turboderp/Mistral-7B-v0.2-exl2'): <coroutine object answer_prompt at 0x7c0885fdd040>}


In [10]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))
pprint(results)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): "The PSF's net income in 2021 was "
                                         '$752 thousand, and in 2022 it was '
                                         '$256 thousand.',
 ('Chroma', 'OpenAI/gpt-4'): 'The net income of PSF in 2021 was $629 thousand '
                             'and in 2022 it was $256 thousand.',
 ('Chroma', 'turboderp/Mistral-7B-v0.2-exl2'): '\n'
                                               '```\n'
                                               '\n'
                                               '## Answer (0)\n'
                                               '\n'
                                               'I think you need to use `\\n` '
                                               'to get a new line.\n'
                                               '\n'
                                               '```\n'
                                               'import openai\n'
                                          

<hr>

**✨ Next: [Conclusion](05-conclusion.ipynb) →**

<hr>